## Data 

Data to test this notebook can be downloaded at https://sdrive.cnrs.fr/s/98EFBQ7EjMR57wJ. Two examples are provided:
- isotropic data, each h5 file contains several frames which should be averaged 
- anisotropic data, corresponding to maps performed on a sample. Detector frames should not be averaged, and integration sectors should be defined.

## Main functionnalities of swing_tools

Swing_tools library can be decomposed in 3 classes:
- Mask: contain methods to perform general tasks on individual files
- SwingData: contain methods 
- SwingBatch: contain methods to perform batch analysis of files contained in a given directory

### Mask Class

The Mask class can be used to generate mask files in a format compatible with the library. It can handle 2 type of files:
- *.txt files, corresponding to maks files generated by foxtrot. The method convertfoxtrotmask() can be used to convert the mask in the suitable 'edf' format
- *.h5 files corresponding to data file from Swing. The method drawmask() uses the scattering data as input of pyFAI-drawmask so that the user can generate its own mask. This function can be useful when using data reconstruction with caving algorithms, in which case a specific mask file should be specified

#### Example of mask creation using h5 data file

Execution of the code below will open pyFAI-drawmask interface. Don't forget to save the mask on your drive before quit.

In [ ]:
from swing_tools import Mask
path='./testdata/'
file=path+'lacroix_04287_2024-07-22_01-31-41.h5'

# create mask using a h5 file
test=Mask(file)
test.draw_mask()


#### Example of conversion of a Foxtrot mask

In [2]:
# On SWING, you will probably get from beamline scientist a mask file in foxtrot format.
# The following code can handle this situation:

foxtrotmaskfile=path+'mask_lacroix_jul24.txt'
mask=Mask(foxtrotmaskfile)
mask.convertfoxtrotmask()

### SwingData class

This class was designed to handle individual h5 files. A SwingData instance can be initialized with the following parameters:
- filename: str : path to data faile (*.h5 or *.nxs)
- maskfile: str : path to mask file (*.edf format)
- basler_coords: tuple:  pixel coordinates of X-ray beam on basler image, provided by the beamline scientist on request (don't forget to ask!)
- basler_calibration: tuple calibration of basler image (in µm/pixels), along X and along Z
- anisotropic_data: bool Set True top optimize integration of anisotropic data (slow down computation), (default= False)
- sectors: List(tuple) List of angular sectors to perform azimuthal integration [(angle, delta)] (default=None, corresponds to isotropic averaging)
- average_frames: bool Tag to perform averaging of scattering frames contained in h5 file (default = True)
- transmission_correction: bool Tag to perform transmission corrections (default=True)

#### List of methods available in SwingData

- save_integration(): This method will save integration results as txt files in a subdirectory called integration
- plot_data(): This method will generate plots of data, stored in a subdirectory called plot
- plot_and_save(): This methods combines the two above mentionned methods in a single command

In [1]:
from swing_tools import SwingData
path='./testdata/'
file=path+'lacroix_04287_2024-07-22_01-31-41.h5'
mask='./testdata/mask.edf'

# For this data, we have several frames in a single h5file, corresponding to a line.
# We therefore won't average the frames contained in a single h5
# We also expect anisotropic behaviour and will integrate horizontally and vertically (up)
# we will use data caving to complete detector gpas, remove beamstop,...
# We don't care about reference subtraction and therefore ignore data normaization by transmission

# Create class instance
data=SwingData(file,
               mask,
               sectors=[(-90,10),(-180,10)],
               basler_coords=[186,309],
               basler_calibration=[3.73,3.67],
               anisotropic_data=True,
               average_frames=False,
               transmission_correction=False)
data.plot_and_save()


### Class SwingBatch

In [ ]:
from swing_tools import SwingBatch
path='./testdata/'
mask='./testdata/mask.edf'
batch=SwingBatch(path,
                 mask,
                 anisotropic_data=True,
                 average_frames=False,
                 transmission_correction=True,
                 basler_coords=(186,309))
batch.process()

In [ ]:
batch.create_video_from_images()